In [1]:
import numpy as np

# AdaBoost

In [5]:
class DecisionStumpDebug:
    """
    Ein Decision Stump mit vollständiger Console-Ausgabe.
    """

    def __init__(self):
        self.threshold = None
        self.polarity = None

    def fit(self, X, y, sample_weight=None):
        X = X.flatten()
        if sample_weight is None:
            sample_weight = np.ones(len(y)) / len(y)

        best_err = 999999
        best_thresh = None
        best_polarity = None

        print("\n=== Decision Stump Training ===")
        print("X:", X)
        print("y:", y)
        print("w:", sample_weight)

        for thresh in np.unique(X):
            for polarity in [+1, -1]:
                preds = np.ones(len(y))
                preds[X * polarity < thresh * polarity] = -1

                err = np.sum(sample_weight[preds != y])

                print(f"  Teste thresh={thresh:.3f}, pol={polarity}, "
                      f"pred={preds}, error={err:.4f}")

                if err < best_err:
                    best_err = err
                    best_thresh = thresh
                    best_polarity = polarity

        print("\n--> Gewählter Stump:")
        print("Threshold:", best_thresh)
        print("Polarity:", best_polarity)
        print("Error:", best_err)
        print("===============================\n")

        self.threshold = best_thresh
        self.polarity = best_polarity

    def predict(self, X):
        X = X.flatten()
        preds = np.ones(len(X))
        preds[X * self.polarity < self.threshold * self.polarity] = -1
        return preds

In [6]:
class AdaBoostDebug:
    def __init__(self, T=5):
        self.T = T
        self.models = []
        self.alphas = []

    def fit(self, X, y):
        n = len(y)
        w = np.ones(n) / n
        
        print("\n================ AdaBoost Training ================\n")
        
        for t in range(1, self.T+1):
            print(f"\n=== Iteration {t} ===")
            print("Aktuelle Gewichte:", np.round(w, 4))

            stump = DecisionStumpDebug()
            stump.fit(X, y, sample_weight=w)
            preds = stump.predict(X)

            err = np.sum(w[preds != y])
            err = max(err, 1e-10)

            alpha = 0.5 * np.log((1 - err) / err)

            print(f"Fehler err_t = {err:.6f}")
            print(f"Alpha_t = {alpha:.6f}")
            print("Predictions:", preds)

            # Gewichte updaten
            w = w * np.exp(-alpha * y * preds)
            print("Un-normalisierte neue Gewichte:", np.round(w, 6))

            w = w / np.sum(w)
            print("Normalisierte neue Gewichte:", np.round(w, 6))

            self.models.append(stump)
            self.alphas.append(alpha)

        print("\n================ Training fertig ================\n")

    def predict(self, X):
        preds = np.zeros(len(X))
        for alpha, stump in zip(self.alphas, self.models):
            preds += alpha * stump.predict(X)
        print("Summenvorhersage F(x):", preds)
        return np.sign(preds)

In [9]:
X = np.array([[1],[2],[3],[4],[5],[6]], dtype=float)
y = np.array([-1,-1,-1,+1,-1,+1], dtype=int)

clf = AdaBoostDebug(T=3)
clf.fit(X, y)
print("== Finale Vorhersage ==")
print(clf.predict(X))


================ AdaBoost Training ================


=== Iteration 1 ===
Aktuelle Gewichte: [0.1667 0.1667 0.1667 0.1667 0.1667 0.1667]

=== Decision Stump Training ===
X: [1. 2. 3. 4. 5. 6.]
y: [-1 -1 -1  1 -1  1]
w: [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
  Teste thresh=1.000, pol=1, pred=[1. 1. 1. 1. 1. 1.], error=0.6667
  Teste thresh=1.000, pol=-1, pred=[ 1. -1. -1. -1. -1. -1.], error=0.5000
  Teste thresh=2.000, pol=1, pred=[-1.  1.  1.  1.  1.  1.], error=0.5000
  Teste thresh=2.000, pol=-1, pred=[ 1.  1. -1. -1. -1. -1.], error=0.6667
  Teste thresh=3.000, pol=1, pred=[-1. -1.  1.  1.  1.  1.], error=0.3333
  Teste thresh=3.000, pol=-1, pred=[ 1.  1.  1. -1. -1. -1.], error=0.8333
  Teste thresh=4.000, pol=1, pred=[-1. -1. -1.  1.  1.  1.], error=0.1667
  Teste thresh=4.000, pol=-1, pred=[ 1.  1.  1.  1. -1. -1.], error=0.6667
  Teste thresh=5.000, pol=1, pred=[-1. -1. -1. -1.  1.  1.], error=0.3333
  Teste thresh=5.000, pol=-1, pred=[ 1.  1.  1. 

---

# AdaBoost und Gradient Boosting: Eine vergleichende Erklärung

Sowohl AdaBoost (Adaptive Boosting) als auch Gradient Boosting (GBM) gehören zu den **Boosting-Algorithmen**, die schwache Lerner (meist Entscheidungsbäume) sequenziell kombinieren, um ein starkes Gesamtmodell zu erzeugen. Der Hauptunterschied liegt jedoch in der Methode, wie sie die Fehler des Vorgängermodells korrigieren.

## 1. AdaBoost: Fokus auf Datenpunktgewichten

AdaBoost wurde entwickelt, bevor das allgemeine Gradient-Boosting-Framework existierte.

* **Mechanismus:** AdaBoost passt die **Gewichtung** der Trainingsdatenpunkte in jeder Iteration an. Falsch klassifizierte Punkte erhalten höhere Gewichte, wodurch der nächste schwache Lerner gezwungen wird, sich auf diese "schwierigen" Beispiele zu konzentrieren.
* **Verlustfunktion:** AdaBoost ist **mathematisch äquivalent** zur schrittweisen Minimierung der **exponentiellen Verlustfunktion** $\mathcal{L}_{\text{exp}}(y, F) = e^{-yF}$.
* **Fazit:** AdaBoost ist **kein** Gradient Boosting im allgemeinen Sinne, aber ein **Spezialfall** des Additiven Modells.


## 2. Gradient Boosting (GBM): Fokus auf Gradienten

GBM ist ein verallgemeinertes Framework, das jede differenzierbare Verlustfunktion verwenden kann.

### Das Prinzip der Pseudo-Residuen

Der nächste schwache Lerner wird nicht auf die Originaldaten oder ihre Gewichte trainiert, sondern auf die **Pseudo-Residuen** ($g_i$):

$$g_i = -\left[\frac{\partial \mathcal{L}(y_i, F_{m-1}(x_i))}{\partial F_{m-1}(x_i)}\right]$$

Der negative Gradient ($\mathbf{-} \nabla \mathcal{L}$) zeigt geometrisch die Richtung des **steilsten Abstiegs** der Verlustfunktion. Das Pseudo-Residuum ist die **optimale Korrektur** (das Residuum), die der nächste Lerner finden muss, um den Gesamtfehler am schnellsten zu verringern.

| Merkmal | **AdaBoost** | **Gradient Boosting (GBM)** |
| :--- | :--- | :--- |
| **Fehlerkorrektur** | Anpassung der **Datenpunktgewichte** | Training auf **Negativem Gradienten** (Pseudo-Residuen) |
| **Verlustfunktion** | Exponentiell (fix) | Beliebig (wählbar) |
| **Sonderfall** | GBM mit exponentieller Verlustfunktion | Allgemeiner Rahmen |

### Die Äquivalenz zum Residuum (Quadrierter Fehler)

Bei der Verwendung des **Quadrierten Fehlers** (Mean Squared Error, MSE) $\mathcal{L}(y, F) = \frac{1}{2}(y - F)^2$ vereinfacht sich der negative Gradient genau zum klassischen Residuum:

$$g_i = - \frac{\partial \mathcal{L}}{\partial F} = y_i - F_{m-1}(x_i)$$

Daher **entspricht** das Pseudo-Residuum in diesem speziellen Fall dem tatsächlichen Residuum.


## Wahl des Schwachen Lerners

* Gradient Boosting verwendet typischerweise **Regressionsbäume** (Regression Trees) als schwache Lerner.
* **Grund:** Die Pseudo-Residuen $g_i$ sind **kontinuierliche (reelle) Werte**, selbst bei Klassifikationsproblemen (dort sind es die Fehler auf der Logit-Skala). Regressionsbäume sind darauf ausgelegt, kontinuierliche Zielwerte optimal vorherzusagen, im Gegensatz zu Klassifikationsbäumen, die diskrete Klassen-Labels vorhersagen.

# GradientBoost

In [23]:
class RegressionStumpDebug:
    """
    Minimaler Regressions-Stump für Gradient Boosting:
    - 1 Feature
    - threshold + konstante Werte (c_left, c_right)
    """

    def __init__(self):
        self.threshold = None
        self.polarity = 1
        self.c_left = None
        self.c_right = None

    def fit(self, X, y, sample_weight=None):
        X = X.flatten()
        y = np.array(y, dtype=float)

        if sample_weight is None:
            sample_weight = np.ones(len(y)) / len(y)

        best_err = np.inf

        print("\n=== Regression Stump Training ===")
        print("X:", X)
        print("Zielwerte (Pseudo-Residuen):", np.round(y, 4))
        print("Gewichte:", np.round(sample_weight, 4))

        for thresh in np.unique(X):
            for polarity in [+1, -1]:

                left = (X * polarity) < (thresh * polarity)
                right = ~left

                if left.sum() == 0 or right.sum() == 0:
                    continue

                w_left = sample_weight[left]
                w_right = sample_weight[right]

                c_left = np.sum(w_left * y[left]) / np.sum(w_left)
                c_right = np.sum(w_right * y[right]) / np.sum(w_right)

                preds = np.where(left, c_left, c_right)
                err = np.sum(sample_weight * (y - preds)**2)

                print(f"  thresh={thresh}, pol={polarity}, "
                      f"c_left={c_left:.3f}, c_right={c_right:.3f}, "
                      f"err={err:.5f}")

                if err < best_err:
                    best_err = err
                    self.threshold = thresh
                    self.polarity = polarity
                    self.c_left = c_left
                    self.c_right = c_right

        print("\n--> Gewählter Regressions-Stump:")
        print("Threshold:", self.threshold)
        print("Polarity:", self.polarity)
        print("c_left:", self.c_left)
        print("c_right:", self.c_right)
        print("Minimaler Fehler:", best_err)
        print("=================================\n")

    def predict(self, X):
        X = X.flatten()
        left = (X * self.polarity) < (self.threshold * self.polarity)
        return np.where(left, self.c_left, self.c_right)


In [24]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [25]:
class LogisticGradientBoostingDebug:

    def __init__(self, T=5, lr=0.5):
        self.T = T
        self.lr = lr
        self.models = []

    def fit(self, X, y):
        y = np.array(y, dtype=float)

        # Start mit log-odds der Grundwahrscheinlichkeit
        p0 = np.mean(y)
        self.F0 = np.log(p0 / (1 - p0))
        F = np.ones(len(y)) * self.F0

        print("\n========== Logistic Gradient Boosting Training ==========")
        print(f"Startwert F0 (log-odds) = {self.F0:.4f}")

        for t in range(1, self.T+1):
            print(f"\n=== Iteration {t} ===")

            p = sigmoid(F)
            r = y - p  # Pseudo-Residuals

            print("Aktuelle Logits F:", np.round(F, 4))
            print("Aktuelle Wahrsch. p:", np.round(p, 4))
            print("Pseudo-Residuen r:", np.round(r, 4))

            stump = RegressionStumpDebug()
            stump.fit(X, r)

            pred = stump.predict(X)
            print("Stump Predictions h_t(x):", np.round(pred, 4))

            F = F + self.lr * pred
            print("Neue Logits F:", np.round(F, 4))

            self.models.append(stump)

        print("\n========= Training beendet =========\n")

    def predict_proba(self, X):
        F = np.ones(len(X)) * self.F0
        for stump in self.models:
            F += self.lr * stump.predict(X)
        return sigmoid(F)

    def predict(self, X):
        return (self.predict_proba(X) > 0.5).astype(int)


In [28]:
X = np.array([[1],[2],[3],[4],[5],[6]], dtype=float)
y = np.array([0,0,0,1,0,1], dtype=int)

gb = LogisticGradientBoostingDebug(T=3, lr=0.5)
gb.fit(X, y)

print("Vorhersagewahrscheinlichkeiten:", np.round(gb.predict_proba(X), 3))
print("Vorhersagen:", gb.predict(X))
print("Wahr:", y)



========== Logistic Gradient Boosting Training ==========
Startwert F0 (log-odds) = -0.6931

=== Iteration 1 ===
Aktuelle Logits F: [-0.6931 -0.6931 -0.6931 -0.6931 -0.6931 -0.6931]
Aktuelle Wahrsch. p: [0.3333 0.3333 0.3333 0.3333 0.3333 0.3333]
Pseudo-Residuen r: [-0.3333 -0.3333 -0.3333  0.6667 -0.3333  0.6667]

=== Regression Stump Training ===
X: [1. 2. 3. 4. 5. 6.]
Zielwerte (Pseudo-Residuen): [-0.3333 -0.3333 -0.3333  0.6667 -0.3333  0.6667]
Gewichte: [0.1667 0.1667 0.1667 0.1667 0.1667 0.1667]
  thresh=1.0, pol=-1, c_left=0.067, c_right=-0.333, err=0.20000
  thresh=2.0, pol=1, c_left=-0.333, c_right=0.067, err=0.20000
  thresh=2.0, pol=-1, c_left=0.167, c_right=-0.333, err=0.16667
  thresh=3.0, pol=1, c_left=-0.333, c_right=0.167, err=0.16667
  thresh=3.0, pol=-1, c_left=0.333, c_right=-0.333, err=0.11111
  thresh=4.0, pol=1, c_left=-0.333, c_right=0.333, err=0.11111
  thresh=4.0, pol=-1, c_left=0.167, c_right=-0.083, err=0.20833
  thresh=5.0, pol=1, c_left=-0.083, c_right=0.1

## Warum unser Gradient Boosting mit Decision Stumps auf diesem Datensatz nicht gut funktioniert

Wir verwenden hier ein sehr kleines und strukturell schwieriges Beispiel:

y = np.array([0,0,0,1,0,1], dtype=int)

Die Zielwerte wechseln also zweimal zwischen 0 und 1:  
drei Nullen → eine Eins → eine Null → eine Eins.

Ein Decision Stump (Baum der Tiefe 1) kann immer nur **eine** Grenze setzen und den Raum in **zwei** Regionen teilen. Damit kann er bestenfalls eine grobe Trennung vornehmen, aber nicht eine Struktur abbilden, in der die Klasse mehrfach hin- und herspringt. Für das Modell sehen die Residuen daher so aus, dass jeder Stump immer nur eine sehr einfache, unzureichende Approximation liefert.

Die Folge ist:  
Auch wenn das Gradient Boosting formal korrekt arbeitet, liefern die Stumps in jeder Iteration **zu wenig Information**, um die Logits sinnvoll in Richtung der richtigen Klassen zu bewegen. Das Modell macht zwar Updates, aber diese reichen nicht aus, um die beiden „eingestreuten“ positiven Beispiele korrekt zu erkennen.

Deshalb führt der Algorithmus zwar mehrere Iterationen aus, aber die Vorhersagen verbessern sich kaum.

Die Lösung besteht darin, den Base Learner etwas stärker zu machen:  
Statt eines Stumps verwenden wir einen Baum mit größerer Tiefe (z. B. depth = 2). Ein solcher Baum kann bereits zwei Splits setzen und damit **drei Regionen** modellieren – genug, um diese einfache, aber nicht stumpf trennbare Struktur zu approximieren.

Darum führen wir den Hyperparameter `max_depth` ein:  
Damit können wir den Base Learner stärker machen und dem Gradient Boosting überhaupt erst ermöglichen, die Struktur des Datensatzes zu „sehen“. Für die Studierenden ist das oft das Aha-Erlebnis: Boosting funktioniert nicht automatisch – die Wahl des Base Learners entscheidet, ob das Modell überhaupt lernfähig ist.

In [47]:
class RegressionTreeDebug:
    """
    Ein generischer Regressionsbaum für Gradient Boosting:
    - beliebige max_depth
    - weighted variance split
    - jeder Leaf = weighted mean
    - komplette Debug-Ausgabe
    """

    class Node:
        def __init__(self, is_leaf=False, value=None,
                     thresh=None, pol=None, left=None, right=None):
            self.is_leaf = is_leaf
            self.value = value
            self.thresh = thresh
            self.pol = pol
            self.left = left
            self.right = right

    def __init__(self, max_depth=1):
        self.max_depth = max_depth
        self.root = None

    def fit(self, X, y, w=None):
        X = X.flatten()
        y = np.array(y, float)
        if w is None:
            w = np.ones_like(y) / len(y)

        print(f"\n====== Train Regression Tree (max_depth={self.max_depth}) ======")
        self.root = self._build_tree(X, y, w, depth=1)
        print("==============================================================\n")

    # -------------------------------------------------------------
    #   Recursive Tree Builder
    # -------------------------------------------------------------
    def _build_tree(self, X, y, w, depth):
        print(f"\n--> Baue Node auf depth={depth}, n={len(y)}")

        # Leaf stopping conditions
        if depth > self.max_depth or len(y) <= 1:
            leaf_value = np.sum(w * y) / np.sum(w)
            print(f"   LEAF: value={leaf_value:.4f}")
            return self.Node(is_leaf=True, value=leaf_value)

        # best split search
        best_err = np.inf
        best_thresh = None
        best_pol = None
        best_left_mask = None
        best_right_mask = None

        print("   Suche besten Split...")

        for thresh in np.unique(X):
            for pol in [1, -1]:
                left_mask = (X * pol) < (thresh * pol)
                right_mask = ~left_mask

                if left_mask.sum() == 0 or right_mask.sum() == 0:
                    continue

                # weighted variance error
                err = (
                    np.sum(w[left_mask]  * (y[left_mask]  - np.mean(y[left_mask]))**2) +
                    np.sum(w[right_mask] * (y[right_mask] - np.mean(y[right_mask]))**2)
                )

                print(f"     Test: thresh={thresh}, pol={pol}, err={err:.5f}")

                if err < best_err:
                    best_err = err
                    best_thresh = thresh
                    best_pol = pol
                    best_left_mask = left_mask
                    best_right_mask = right_mask

        # if no split works => leaf
        if best_thresh is None:
            leaf_value = np.sum(w * y) / np.sum(w)
            print(f"   (keine sinnvollen Splits) LEAF={leaf_value:.4f}")
            return self.Node(is_leaf=True, value=leaf_value)

        print(f"   -> Bester Split: thresh={best_thresh}, pol={best_pol}, err={best_err:.5f}")

        # recursively build children
        left_node = self._build_tree(X[best_left_mask],
                                     y[best_left_mask],
                                     w[best_left_mask],
                                     depth + 1)

        right_node = self._build_tree(X[best_right_mask],
                                      y[best_right_mask],
                                      w[best_right_mask],
                                      depth + 1)

        return self.Node(is_leaf=False,
                         thresh=best_thresh,
                         pol=best_pol,
                         left=left_node,
                         right=right_node)

    # -------------------------------------------------------------
    #   Prediction
    # -------------------------------------------------------------
    def _predict_one(self, x, node):
        if node.is_leaf:
            return node.value

        if (x * node.pol) < (node.thresh * node.pol):
            return self._predict_one(x, node.left)
        else:
            return self._predict_one(x, node.right)

    def predict(self, X):
        X = X.flatten()
        return np.array([self._predict_one(x, self.root) for x in X])


In [31]:
class LogisticGradientBoostingDebug:

    def __init__(self, T=5, lr=0.5, max_depth=1):
        """
        T         = Anzahl der Boosting-Stufen
        lr        = Lernrate
        max_depth = Maximal-Tiefe des Base Learners (Regressionsbaum)
        """
        self.T = T
        self.lr = lr
        self.max_depth = max_depth
        self.models = []

    def fit(self, X, y):
        y = np.array(y, dtype=float)

        # Start-F0 = log-odds der Grundwahrscheinlichkeit
        p0 = np.mean(y)
        self.F0 = np.log(p0 / (1 - p0))
        F = np.ones(len(y)) * self.F0

        print("\n========== Logistic Gradient Boosting Training ==========")
        print(f"Startwert F0 (log-odds) = {self.F0:.4f}")
        print(f"Base Learner max_depth = {self.max_depth}\n")

        for t in range(1, self.T+1):
            print(f"\n=== Iteration {t} ===")

            # probabilistische predictions
            p = sigmoid(F)
            r = y - p

            print("Aktuelle Logits F:", np.round(F, 4))
            print("Aktuelle Wahrsch. p:", np.round(p, 4))
            print("Pseudo-Residuen r:", np.round(r, 4))

            # --- HIER wird der Base-Learner mit max_depth erzeugt ---
            stump = RegressionTreeDebug(max_depth=self.max_depth)
            stump.fit(X, r)

            pred = stump.predict(X)
            print("h_t(x):", np.round(pred, 4))

            F = F + self.lr * pred
            print("Neue Logits F:", np.round(F, 4))

            self.models.append(stump)

        print("\n========= Training beendet =========\n")

    def predict_proba(self, X):
        F = np.ones(len(X)) * self.F0
        for stump in self.models:
            F += self.lr * stump.predict(X)
        return sigmoid(F)

    def predict(self, X):
        return (self.predict_proba(X) > 0.5).astype(int)


In [33]:
X = np.array([[1],[2],[3],[4],[5],[6]])
y = np.array([0,0,0,1,0,1])

gb = LogisticGradientBoostingDebug(
    T=10,
    lr=0.5,
    max_depth=2
)

gb.fit(X, y)

print("Pred:", gb.predict(X))
print("True:", y)


========== Logistic Gradient Boosting Training ==========
Startwert F0 (log-odds) = -0.6931
Base Learner max_depth = 2


=== Iteration 1 ===
Aktuelle Logits F: [-0.6931 -0.6931 -0.6931 -0.6931 -0.6931 -0.6931]
Aktuelle Wahrsch. p: [0.3333 0.3333 0.3333 0.3333 0.3333 0.3333]
Pseudo-Residuen r: [-0.3333 -0.3333 -0.3333  0.6667 -0.3333  0.6667]

====== Train Regression Tree (max_depth=2) ======

--> Baue Node auf depth=1, n=6
   Suche besten Split...
     Test: thresh=1, pol=-1, err=0.20000
     Test: thresh=2, pol=1, err=0.20000
     Test: thresh=2, pol=-1, err=0.16667
     Test: thresh=3, pol=1, err=0.16667
     Test: thresh=3, pol=-1, err=0.11111
     Test: thresh=4, pol=1, err=0.11111
     Test: thresh=4, pol=-1, err=0.20833
     Test: thresh=5, pol=1, err=0.20833
     Test: thresh=5, pol=-1, err=0.13333
     Test: thresh=6, pol=1, err=0.13333
   -> Bester Split: thresh=3, pol=-1, err=0.11111

--> Baue Node auf depth=2, n=3
   Suche besten Split...
     Test: thresh=4, pol=-1, err=0.

# Gradient Boosting vs. XGBoost: Eine vergleichende Erklärung

Gradient Boosting (GBM) und XGBoost gehören zur gleichen Familie: Beide sind **additive Modelle**, die schwache Lerner (normalerweise Entscheidungsbäume) schrittweise aufbauen, indem sie Fehler der vorherigen Iterationen korrigieren.  
XGBoost ist jedoch **nicht einfach „Gradient Boosting, aber schneller“** – es erweitert das klassische Gradient Boosting um wichtige Verbesserungen in Optimierung, Regularisierung und Robustheit.

## 1. Gradient Boosting: Optimierung über negative Gradienten

Gradient Boosting ist das allgemeine Framework.  
In jeder Iteration wird ein schwacher Lerner auf den **Pseudo-Residuen** trainiert:

$$g_i = -\frac{\partial \mathcal{L}(y_i, F(x_i))}{\partial F(x_i)}.$$

Der Baum versucht optimal abzubilden, wie der Gesamtfehler (Loss) am stärksten reduziert werden kann.

### Zentrale Eigenschaften von klassischem GBM
* Bäume approximieren **nur** den **1. Ableitungsfehler** (negative Gradienten).
* Keine explizite Regularisierung in der Loss-Form.
* Leaf-Werte werden als einfacher gewichteter Mittelwert der Residuen bestimmt.
* Splits basieren auf **Fehlerreduktion** (z. B. Varianzreduktion oder Entropieersparnis).
* GBM ist rein funktional auf das Minimieren der Loss ausgerichtet – ohne strukturelle Nebenbedingungen.

Gradient Boosting ist dadurch flexibel, aber empfindlich gegenüber:
- Rauschen,  
- zu tiefen Bäumen,  
- komplexen Residuen,  
- Overfitting auf kleinen Datensätzen.


## 2. XGBoost: Gradient Boosting mit 2. Ableitung, Regularisierung und Optimierung

XGBoost erweitert das klassische GBM in drei wesentlichen Dimensionen:

### (a) Optimierung mit 2. Ordnung (Newton-Schritt)
XGBoost nutzt nicht nur den ersten Gradienten \(g_i\), sondern auch die **zweite Ableitung** (Hessian):

$$h_i = \frac{\partial^2 \mathcal{L}}{\partial F(x_i)^2}.$$

Damit stellt XGBoost eine **zweite Ordnung Taylor-Approximation** des Loss bereit.  
Die Leaf-Werte werden nach einer exakten Formel berechnet:

$$
w = -\frac{\sum_i g_i}{\sum_i h_i + \lambda}
$$

Das bedeutet:
- tiefere, „präzisere“ Updates
- bessere Konvergenz
- stabilere Trainingsschritte

Das ist wesentlich genauer als die „Mittelwert-Update“-Logik von einfachem Gradient Boosting.


### (b) Struktur- und Gewichtregularisierung
XGBoost reguliert das Baumwachstum explizit.  
Die Trainingsobjective enthält zusätzlich:

- **L2-Regularisierung** der Leaf-Werte \( \lambda w^2 \)
- **L1-Regularisierung** (optional)
- **Penalty auf die Anzahl der Leaves** (γ)

Damit kontrolliert XGBoost Überanpassung bereits **während** des Baumwachstums.

Gradient Boosting geregelt: nur über Lernrate & Baumtiefe  
XGBoost geregelt: zusätzlich Leaf-Regularisierung, Strukturkosten, Child-Weight-Bedingungen


### (c) Strikte Split-Kriterien
Ein Split wird nur akzeptiert, wenn der Gain:

$$
\text{Gain} > 0
$$

ist.  
Der Gain beruht auf der 2. Ordnung und enthält Regularisierung:

$$
\text{Gain} = \frac{1}{2}
\left[
   \frac{G_L^2}{H_L+\lambda}
 + \frac{G_R^2}{H_R+\lambda}
 - \frac{(G_L + G_R)^2}{H_L + H_R + \lambda}
\right] - \gamma
$$

Damit ignoriert XGBoost alle Splits, die nicht wirklich nützlich sind.  
Gradient Boosting splitet dagegen rein auf Residuenoptimierung (ohne solche Hürden).


### (d) Leaf-Gewichte werden über Newton-Schritte aktualisiert
GBM:  
Leaf-Wert = Mittelwert der Residuen (nur 1. Ordnung)

XGBoost:  
Leaf-Wert = geschlossener Newton-Schritt (1. + 2. Ordnung)

Das ergibt oft deutlich stabilere Updates.


## 3. Direkter Vergleich

| Merkmal | **Gradient Boosting (GBM)** | **XGBoost** |
|--------|------------------------------|--------------|
| Optimierungsverfahren | 1. Ableitung (Gradient) | 1. + 2. Ableitung (Newton-Schritt) |
| Leaf-Berechnung | Mittelwert der Residuen | Geschlossene Formel mit Hessian und λ |
| Overfitting-Kontrolle | nur über Lernrate/Depth | L1/L2-Regularisierung, γ-Penalty, Child-Weight |
| Split-Kriterien | reine Fehlerreduktion | Gain-Formel mit Regularisierung |
| Umgang mit kleinen Leafs | erlaubt | verhindert, wenn Hessians zu klein |
| Stabilität | anfälliger | robuster durch 2. Ordnung |
| Geschwindigkeit | oft langsamer | optimierte Implementierung (nicht nur Theorie) |


## 4. Kernaussage

Gradient Boosting ist ein allgemeiner Ansatz: „Baue das Modell, indem du Residuen annäherst.“

XGBoost ist die verfeinerte, regulierte, stabilisierte und Newton-optimierte Version davon:  
„Baue das Modell effizient, streng reguliert, mit 1. + 2. Ableitung, um Überfitting und instabile Updates zu vermeiden.“

Man kann es so zusammenfassen:

**GBM = Gradient-Schritt**  
**XGBoost = Newton-Schritt + Regularisierung + Optimierter Baumaufbau**

Damit ist XGBoost eine direkte Weiterentwicklung des klassischen Gradient Boosting – mit klaren Vorteilen in Stabilität, Genauigkeit und Robustheit.


# XG Boost

In [35]:
class XGBTree:
    class Node:
        def __init__(self, is_leaf=False, value=None,
                     feature=None, thresh=None, left=None, right=None):
            self.is_leaf = is_leaf
            self.value = value
            self.feature = feature
            self.thresh = thresh
            self.left = left
            self.right = right

    def __init__(self, max_depth=3, lambda_=1.0, min_child_weight=1.0):
        self.max_depth = max_depth
        self.lambda_ = lambda_
        self.min_child_weight = min_child_weight
        self.root = None

    # ---------------------------------------------------------
    #           FIT (Newton, Gain-basiert)
    # ---------------------------------------------------------
    def fit(self, X, g, h):
        X = np.array(X)
        self.root = self._build_tree(X, g, h, depth=1)
        return self

    # ---------------------------------------------------------
    #         Bilde einen Baum rekursiv
    # ---------------------------------------------------------
    def _build_tree(self, X, g, h, depth):

        print(f"\n--- Build node: depth={depth}, n={len(X)} ---")

        # Leaf criteria
        if depth > self.max_depth or len(X) <= 1 or np.sum(h) < self.min_child_weight:
            leaf_value = -np.sum(g) / (np.sum(h) + self.lambda_)
            print(f"  Leaf: value={leaf_value:.4f}")
            return self.Node(is_leaf=True, value=leaf_value)

        best_gain = 0
        best_feature = None
        best_thresh = None
        best_masks = None

        # Try all features + thresholds
        for feature in range(X.shape[1]):
            xs = X[:, feature]

            for thresh in np.unique(xs):
                left = xs < thresh
                right = ~left

                if left.sum() == 0 or right.sum() == 0:
                    continue

                # G_L, H_L
                GL = np.sum(g[left])
                HL = np.sum(h[left])

                # G_R, H_R
                GR = np.sum(g[right])
                HR = np.sum(h[right])

                # Gain (XGBoost original formula)
                gain = 0.5 * (
                    (GL**2) / (HL + self.lambda_) +
                    (GR**2) / (HR + self.lambda_) -
                    ( (GL+GR)**2 ) / ( (HL+HR) + self.lambda_)
                )

                print(f"  Test split: x[{feature}] < {thresh}, gain={gain:.6f}")

                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_thresh = thresh
                    best_masks = (left, right)

        # If no positive gain → leaf
        if best_feature is None:
            leaf_value = -np.sum(g) / (np.sum(h) + self.lambda_)
            print(f"  No positive gain → Leaf={leaf_value:.4f}")
            return self.Node(is_leaf=True, value=leaf_value)

        print(f"\n  >>> Best split: feature={best_feature}, thresh={best_thresh}, gain={best_gain:.6f}")

        left_mask, right_mask = best_masks

        left_child = self._build_tree(X[left_mask],
                                      g[left_mask],
                                      h[left_mask],
                                      depth + 1)

        right_child = self._build_tree(X[right_mask],
                                       g[right_mask],
                                       h[right_mask],
                                       depth + 1)

        return self.Node(
            is_leaf=False,
            feature=best_feature,
            thresh=best_thresh,
            left=left_child,
            right=right_child
        )

    # ---------------------------------------------------------
    #          Predict for one sample
    # ---------------------------------------------------------
    def _predict_one(self, x, node):
        if node.is_leaf:
            return node.value
        if x[node.feature] < node.thresh:
            return self._predict_one(x, node.left)
        return self._predict_one(x, node.right)

    def predict(self, X):
        return np.array([self._predict_one(x, self.root) for x in X])

In [36]:
class XGBoostClassifierDebug:

    def __init__(self, T=10, lr=0.3, max_depth=3, lambda_=1.0, min_child_weight=1.0):
        self.T = T
        self.lr = lr
        self.max_depth = max_depth
        self.lambda_ = lambda_
        self.min_child_weight = min_child_weight
        self.trees = []
        self.F0 = 0  # initial logits

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)

        # initial probability
        p0 = np.mean(y)
        self.F0 = np.log(p0 / (1 - p0))
        F = np.ones(len(y)) * self.F0

        print(f"\n========= XGBoost Training Start =========")
        print(f"Initial F0 = {self.F0:.4f}\n")

        for t in range(self.T):
            print(f"\n=== Iteration {t+1} ===")

            # probabilities
            p = sigmoid(F)

            # 1st order gradient (derivative of logloss)
            g = p - y

            # 2nd order gradient (Hessian)
            h = p * (1 - p)

            print("g:", np.round(g, 4))
            print("h:", np.round(h, 4))

            # train tree on (g, h)
            tree = XGBTree(
                max_depth=self.max_depth,
                lambda_=self.lambda_,
                min_child_weight=self.min_child_weight
            )
            tree.fit(X, g, h)

            # update logits
            pred = tree.predict(X)
            print("Tree predictions:", np.round(pred, 4))

            F += self.lr * pred
            print("Neue Logits F:", np.round(F, 4))

            self.trees.append(tree)

        print("\n========= Training fertig =========\n")

    def predict_proba(self, X):
        X = np.array(X)
        F = np.ones(len(X)) * self.F0
        for tree in self.trees:
            F += self.lr * tree.predict(X)
        return sigmoid(F)

    def predict(self, X):
        return (self.predict_proba(X) > 0.5).astype(int)


In [48]:
X = np.array([[1],[2],[3],[4],[5],[6]])
y = np.array([0,0,0,1,0,1])

xgb = XGBoostClassifierDebug(
    T=10,
    lr=0.4,
    max_depth=3,
    lambda_=1.0,
    min_child_weight=1.0
)

xgb.fit(X, y)

print("Pred:", xgb.predict(X))
print("True:", y)
print("Proba:", np.round(xgb.predict_proba(X), 3))



========= XGBoost Training Start =========
Initial F0 = -0.6931


=== Iteration 1 ===
g: [ 0.3333  0.3333  0.3333 -0.6667  0.3333 -0.6667]
h: [0.2222 0.2222 0.2222 0.2222 0.2222 0.2222]

--- Build node: depth=1, n=6 ---
  Test split: x[0] < 2, gain=0.143541
  Test split: x[0] < 3, gain=0.542986
  Test split: x[0] < 4, gain=1.200000
  Test split: x[0] < 5, gain=0.135747
  Test split: x[0] < 6, gain=0.574163

  >>> Best split: feature=0, thresh=4, gain=1.200000

--- Build node: depth=2, n=3 ---
  Leaf: value=-0.6000

--- Build node: depth=2, n=3 ---
  Leaf: value=0.6000
Tree predictions: [-0.6 -0.6 -0.6  0.6  0.6  0.6]
Neue Logits F: [-0.9331 -0.9331 -0.9331 -0.4531 -0.4531 -0.4531]

=== Iteration 2 ===
g: [ 0.2823  0.2823  0.2823 -0.6114  0.3886 -0.6114]
h: [0.2026 0.2026 0.2026 0.2376 0.2376 0.2376]

--- Build node: depth=1, n=6 ---
  Test split: x[0] < 2, gain=0.100506
  Test split: x[0] < 3, gain=0.385772
  Test split: x[0] < 4, gain=0.852243
  Test split: x[0] < 5, gain=0.063619
  

## Warum unser selbst gebautes XGBoost mit 100 Iterationen das einfache Beispiel nicht perfekt löst

Auf den ersten Blick wirkt es seltsam: Wir haben ein sehr kleines, übersichtliches Datenset (nur 6 Punkte, 1 Feature), 100 Boosting-Iterationen, und trotzdem klassifiziert unser XGBoost-Klon einen Punkt weiterhin falsch. Das ist ein guter Anlass, genauer hinzuschauen, was im Inneren des Algorithmus passiert.

### 1. Die Implementierung ist prinzipiell korrekt – aber zu „streng“ für Minidaten

Die Grundidee unseres Codes stimmt:

- Wir arbeiten mit Log-Loss und Logits \(F(x)\).
- Wir berechnen erste und zweite Ableitung der Loss:
  - Gradient \(g_i = p_i - y_i\)
  - Hessian \(h_i = p_i(1 - p_i)\)
- Wir lernen Bäume, deren Leaf-Werte nach der XGBoost-Formel bestimmt werden:
  \[
  w = -\frac{\sum g_i}{\sum h_i + \lambda}
  \]
- Wir wählen Splits über einen Gain:
  \[
  \text{Gain} = \frac{1}{2} \left( \frac{G_L^2}{H_L + \lambda} + \frac{G_R^2}{H_R + \lambda} - \frac{(G_L+G_R)^2}{H_L+H_R+\lambda} \right)
  \]

Das ist inhaltlich XGBoost-Logik. Das Problem liegt nicht in der Theorie, sondern in der Kombination aus:

- extrem kleinem Datensatz,
- konserviven Default-Hyperparametern und
- strengen Abbruchkriterien.

### 2. Warum unsere Bäume kaum noch etwas lernen

Entscheidend sind drei Punkte:

1. **Gain-Schwelle**  
   Wir initialisieren `best_gain` mit 0 und akzeptieren nur Splits, deren Gain > 0 ist.  
   Bei nur 6 Datenpunkten sind die Gradienten- und Hessian-Summen sehr klein. Dadurch wird der berechnete Gain häufig nur minimal positiv oder sogar leicht negativ.  
   Ergebnis: Oft findet der Baum keinen Split mit Gain > 0 und wird direkt zu einem Leaf.

2. **L2-Regularisierung (\(\lambda\))**  
   In der Leaf-Formel taucht \(\lambda\) im Nenner auf:
   \[
   w = -\frac{G}{H + \lambda}
   \]
   Wenn \(H\) (Summe der Hessians) klein ist und \(\lambda\) relativ groß gewählt ist (z. B. 1.0), dann wird \(w\) sehr stark „klein gedrückt“.  
   Die Bäume erzeugen dann Leaf-Werte, die nur winzige Updates machen. Selbst mit vielen Iterationen verschiebt sich \(F(x)\) nur langsam.

3. **`min_child_weight` und kleine Leaves**  
   XGBoost bricht Splits ab, wenn die Summe der Hessians in einem Kind-Knoten kleiner als `min_child_weight` ist. Bei 6 Datenpunkten und Log-Loss liegen die Hessians schnell im Bereich 0.1–0.25. Die Summe pro Leaf ist dann klein.  
   Wenn `min_child_weight` zu hoch gewählt ist, werden viele Splits gar nicht mehr erlaubt, weil die Leaves als „zu klein / zu unsicher“ eingestuft werden.

In Kombination führen diese Mechanismen dazu, dass die meisten Iterationen nur Bäume mit „fast nichts“ lernen:

- Entweder sie werden sofort zu einem Leaf mit Wert nahe 0,
- oder sie produzieren extrem kleine Leaf-Werte,
- oder sie splitten kaum, weil `min_child_weight` den Split blockiert.

Das erklärt den beobachteten Verlauf: Die Wahrscheinlichkeiten bewegen sich langsam, aber kommen nicht so klar über die 0.5-Schwelle, dass alle Punkte korrekt klassifiziert werden.

### 3. Warum das bei echten XGBoost-Setups kaum auffällt

Auf großen Datensätzen mit vielen Beobachtungen:

- sind die Gradienten- und Hessian-Summen größer,
- ist der Gain deutlich von 0 unterscheidbar,
- wirken Regularisierung und `min_child_weight` als sinnvolle Stabilisierung.

Auf Minidatensätzen hingegen dominiert die Regularisierung alles und die eingebauten „Sicherheitsmechanismen“ sorgen dafür, dass der Baum lieber gar nicht splittet, als einen potentiell instabilen Split mit wenig Daten zu machen.

### 4. Was man ändern müsste, damit das Beispiel „sichtbar“ funktioniert

Für didaktische Zwecke (kleines 1D-Spielbeispiel) kann man die Implementierung und Hyperparameter „entschärfen“, zum Beispiel:

- `best_gain` nicht bei 0, sondern bei \(-\infty\) initialisieren, damit immer irgendein Split gewählt wird.
- \(\lambda\) stark reduzieren oder auf 0 setzen, damit die Leaf-Werte nicht weggedämpft werden.
- `min_child_weight` auf 0 setzen, damit auch kleine Leaves zugelassen werden.
- Optional: Falls kein Split mit positivem Gain gefunden wird, einen einfachen Fallback-Split (z. B. Median-Split) verwenden, statt sofort ein Leaf zu erzeugen.

Dann verhält sich der Algorithmus auf diesem Spielzeugbeispiel viel „aggressiver“ und man sieht, wie er die Struktur der 6 Punkte tatsächlich lernt.

### 5. Zusammenfassung

Die Implementierung zeigt sehr gut:

- XGBoost ist nicht immer einfach gut – es ist ein Gradient-Boosting-Schema mit vielen Stabilitäts-Tricks.
- Auf sehr kleinen Datensätzen können genau diese Tricks dazu führen, dass nichts mehr passiert.
- Die Hyperparameter (`lambda`, `min_child_weight`, Tiefe, Gain-Schwelle) sind nicht nur Feintuning, sondern steuern direkt, ob das Modell überhaupt noch lernt.

Das erklärt, warum unser selbstgeschriebener XGBoost mit 100 Iterationen auf 6 Punkten nicht perfekt wird – und welche Stellschrauben man (für Lernzwecke) lockern müsste, damit er es tut.

In [41]:
class XGBTree:
    class Node:
        def __init__(self, is_leaf=False, value=None,
                     feature=None, thresh=None, left=None, right=None):
            self.is_leaf = is_leaf
            self.value = value
            self.feature = feature
            self.thresh = thresh
            self.left = left
            self.right = right

    def __init__(self, max_depth=3, lambda_=1.0, min_child_weight=1.0):
        self.max_depth = max_depth
        self.lambda_ = lambda_
        self.min_child_weight = min_child_weight
        self.root = None

    # ---------------------------------------------------------
    #           FIT (Newton, Gain-basiert)
    # ---------------------------------------------------------
    def fit(self, X, g, h):
        X = np.array(X)
        self.root = self._build_tree(X, g, h, depth=1)
        return self

    # ---------------------------------------------------------
    #         Bilde einen Baum rekursiv
    # ---------------------------------------------------------
    def _build_tree(self, X, g, h, depth):

        print(f"\n--- Build node: depth={depth}, n={len(X)} ---")

        # Leaf criteria
        if depth > self.max_depth or len(X) <= 1 or np.sum(h) < self.min_child_weight:
            leaf_value = -np.sum(g) / (np.sum(h) + self.lambda_)
            print(f"  Leaf: value={leaf_value:.4f}")
            return self.Node(is_leaf=True, value=leaf_value)

        best_gain = -np.inf
        best_feature = None
        best_thresh = None
        best_masks = None

        # Try all features + thresholds
        for feature in range(X.shape[1]):
            xs = X[:, feature]

            for thresh in np.unique(xs):
                left = xs < thresh
                right = ~left

                if left.sum() == 0 or right.sum() == 0:
                    continue

                # G_L, H_L
                GL = np.sum(g[left])
                HL = np.sum(h[left])

                # G_R, H_R
                GR = np.sum(g[right])
                HR = np.sum(h[right])

                # Gain (XGBoost original formula)
                gain = (
                    (GL**2)/(HL + self.lambda_) +
                    (GR**2)/(HR + self.lambda_) -
                    ((GL+GR)**2)/( (HL+HR) + self.lambda_)
                )

                print(f"  Test split: x[{feature}] < {thresh}, gain={gain:.6f}")

                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_thresh = thresh
                    best_masks = (left, right)

        # If no positive gain → leaf
        if best_feature is None:
            print("  Kein Split mit Gain > 0 — Fallback-Split wird verwendet.")
        
            # Fallback: median-split auf Feature 0
            best_feature = 0
            best_thresh = np.median(X[:, 0])
        
            left = X[:,0] < best_thresh
            right = ~left
        
            # Falls sogar das failt → Leaf
            if left.sum() == 0 or right.sum() == 0:
                leaf_value = -np.sum(g) / (np.sum(h) + self.lambda_)
                print(f"  Selbst Fallback nicht möglich → Leaf={leaf_value:.4f}")
                return self.Node(is_leaf=True, value=leaf_value)
        
            best_masks = (left, right)
        
            print(f"  Fallback-Split: feature={best_feature}, thresh={best_thresh}")

        print(f"\n  >>> Best split: feature={best_feature}, thresh={best_thresh}, gain={best_gain:.6f}")

        left_mask, right_mask = best_masks

        left_child = self._build_tree(X[left_mask],
                                      g[left_mask],
                                      h[left_mask],
                                      depth + 1)

        right_child = self._build_tree(X[right_mask],
                                       g[right_mask],
                                       h[right_mask],
                                       depth + 1)

        return self.Node(
            is_leaf=False,
            feature=best_feature,
            thresh=best_thresh,
            left=left_child,
            right=right_child
        )

    # ---------------------------------------------------------
    #          Predict for one sample
    # ---------------------------------------------------------
    def _predict_one(self, x, node):
        if node.is_leaf:
            return node.value
        if x[node.feature] < node.thresh:
            return self._predict_one(x, node.left)
        return self._predict_one(x, node.right)

    def predict(self, X):
        return np.array([self._predict_one(x, self.root) for x in X])

In [45]:
X = np.array([[1],[2],[3],[4],[5],[6]])
y = np.array([0,0,0,1,0,1])

xgb = XGBoostClassifierDebug(
    T=10,
    lr=0.4,
    max_depth=3,
    lambda_=0.0,
    min_child_weight=0.0
)

xgb.fit(X, y)

print("Pred:", xgb.predict(X))
print("True:", y)
print("Proba:", np.round(xgb.predict_proba(X), 3))


========= XGBoost Training Start =========
Initial F0 = -0.6931


=== Iteration 1 ===
g: [ 0.3333  0.3333  0.3333 -0.6667  0.3333 -0.6667]
h: [0.2222 0.2222 0.2222 0.2222 0.2222 0.2222]

--- Build node: depth=1, n=6 ---
  Test split: x[0] < 2, gain=0.600000
  Test split: x[0] < 3, gain=1.500000
  Test split: x[0] < 4, gain=3.000000
  Test split: x[0] < 5, gain=0.375000
  Test split: x[0] < 6, gain=2.400000

  >>> Best split: feature=0, thresh=4, gain=3.000000

--- Build node: depth=2, n=3 ---
  Test split: x[0] < 2, gain=0.000000
  Test split: x[0] < 3, gain=0.000000

  >>> Best split: feature=0, thresh=2, gain=0.000000

--- Build node: depth=3, n=1 ---
  Leaf: value=-1.5000

--- Build node: depth=3, n=2 ---
  Test split: x[0] < 3, gain=0.000000

  >>> Best split: feature=0, thresh=3, gain=0.000000

--- Build node: depth=4, n=1 ---
  Leaf: value=-1.5000

--- Build node: depth=4, n=1 ---
  Leaf: value=-1.5000

--- Build node: depth=2, n=3 ---
  Test split: x[0] < 5, gain=0.750000
  Tes